# Chronic Kidney Disease
Data has 25 feattures which may predict a patient with chronic kidney disease

In [ ]:
pip install -U scikit-learn


![](https://i.pinimg.com/564x/d8/ad/c5/d8adc5543f0f47ebb4e7da821f428937.jpg)

## Index

1. Data Description
2. Used Python Libraries
3. Know Dataset Nature
4. Exploratory data analysis (EDA)
5. Data Preprocessing
6. Data Normalization
7. Feature Selection
8. Feature engineering
9. Model Buliding
10. Receiver Operating Characteristic Curve (ROC AUC)
11. conclusion

## Data Description:

We use the following representation to collect the dataset

1. age - age
2. bp - blood pressure
3. sg - specific gravity
4. al - albumin
5. su - sugar
6. rbc - red blood cells
7. pc - pus cell
8. pcc - pus cell clumps
9. ba - bacteria
10. bgr - blood glucose random
11. bu - blood urea
12. sc - serum creatinine
13. sod - sodium
14. pot - potassium
15. hemo - hemoglobin
16. pcv - packed cell volume
17. wc - white blood cell count
18. rc - red blood cell count
19. htn - hypertension
20. dm - diabetes mellitus
21. cad - coronary artery disease
22. appet - appetite
23. pe - pedal edema
24. ane - anemia
25. class - class

## Used Python Libraries

In [ ]:
from sklearn.metrics import confusion_matrix ,classification_report,precision_score, recall_score ,f1_score, roc_curve, roc_auc_score 
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression 
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC

import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')



## Know Dataset Nature

In [ ]:
data = pd.read_csv('/kaggle/input/ckdisease/kidney_disease.csv')

In [ ]:
data.head()

In [ ]:
data.tail()

In [ ]:
data.info()

In [ ]:
data.describe()

## Exploratory data analysis (EDA)

1) For numeric data

       Made histograms to understand distributions
       Corrplot
       Pivot table comparing survival rate across numeric variables
2) For Categorical Data

       Made bar charts to understand balance of classes
       Made pivot tables to understand relationship with survival

In [ ]:
data_num = data[['age','bp','sg','al','su','bgr','bu','sc','sod','pot','hemo']]
data_cat = data[['rbc','pc','pcc','ba','pcv','wc','rc','htn','dm','cad','appet','pe','ane']]

In [ ]:
filna = data[['id', 'age', 'bp', 'sg', 'al', 'su', 'rbc', 'pc', 'pcc', 'ba', 'bgr',
       'bu', 'sc', 'sod', 'pot', 'hemo', 'pcv', 'wc', 'rc', 'htn', 'dm', 'cad',
       'appet', 'pe', 'ane', 'classification']]


In [ ]:
for i in data_num.columns:
    plt.hist(data_num[i])
    plt.title(i)
    plt.show()

In [ ]:
print(data_num.corr())
sns.heatmap(data_num.corr())

In [ ]:
pd.pivot_table(data, index='classification', values=['age','bp','sg','al','su','bgr','bu','sc','sod','pot','hemo'])

In [ ]:
for i in data_cat.columns:
    sns.barplot(data_cat[i].value_counts().index,data_cat[i].value_counts()).set_title(i)
    plt.show()

In [ ]:
data_cat = data[['rbc','pc','pcc','ba','pcv','wc','rc','htn','dm','cad','appet','pe','ane']]

In [ ]:
print(data.classification.value_counts())
data.classification.replace("ckd\t","ckd",inplace=True)
print(data.classification.value_counts())
print("==="*20)
print(data.dm.value_counts())
data.dm.replace(["\tno","\tyes"," yes"],["no","yes","yes"],inplace=True)
print(data.dm.value_counts())
print("==="*20)
print(data.cad.value_counts())
data.cad.replace(["\tno"],["no"],inplace=True)
print(data.cad.value_counts())

In [ ]:
for i in data_cat:
    print(pd.pivot_table(data,index='classification',columns=i, values='age'))
    print("=="*20)

## Data Preprocessing
1. deal with unwanted text.
2. deal with missing values
3. histogram
4. heatmap
5. pivot_table

In [ ]:
data.rc.replace("\t?",data.rc.mode()[0], inplace=True)
data.rc = data.rc.apply(lambda x: float(x))

data.wc.replace("\t?",data.wc.mode()[0], inplace=True)
data.wc = data.wc.apply(lambda x: float(x))


data.pcv.replace(["\t?","\t43"],data.pcv.mode()[0], inplace=True)
data.pcv = data.pcv.apply(lambda x: float(x))

data.classification.replace(["ckd","notckd"],[1,0], inplace=True)

In [ ]:
data_final_num = data[['age','bp','sg','al','su','bgr','bu','sc','sod','pot','hemo','pcv','wc','rc']]
data_fincal_cat = data[['rbc','pc','pcc','ba','htn','dm','cad','appet','pe','ane']]

In [ ]:
for i in data_final_num.columns:
    plt.hist(data_final_num[i])
    plt.title(i)
    plt.show()

In [ ]:
sns.heatmap(data_final_num.corr())

In [ ]:
pd.pivot_table(data, index='classification', values=['age','bp','sg','al','su','bgr','bu','sc','sod','pot','hemo','pcv','wc','rc'])

In [ ]:
for i in data_fincal_cat:
    print(pd.pivot_table(data,index='classification',columns=i, values='age'))
    print("=="*20)

In [ ]:
# dealing with missing values
for i in filna.columns:
    if data[i].isna().sum() > 0 :
        if data[i].dtype == 'float64':
            data[i].fillna(data[i].median(), inplace=True)
        else:
            data[i].fillna(data[i].mode()[0], inplace=True)


In [ ]:
# dealing with outliers values
for i in data_final_num.columns:
    sns.boxplot(data_final_num[i])
    plt.title(i)
    plt.show()


## Data Normalization

1. percentile(interquartile range)
2. boxplot     

In [ ]:
def outlinefree(dataCol):    
    sorted(dataCol)
    Q1,Q3 = np.percentile(dataCol,[25,75])   
    IQR = Q3-Q1   
    LowerRange = Q1-(1.5 * IQR)   
    UpperRange = Q3+(1.5 * IQR)   
    return LowerRange,UpperRange

In [ ]:
Lowage,Upage = outlinefree(data.age)
Lowbp,Upbp = outlinefree(data.bp)
Lowsg,Upsg = outlinefree(data.sg)
Lowal,Upal = outlinefree(data.al)
Lowsu,Upsu = outlinefree(data.su)
Lowbgr,Upbgr = outlinefree(data.bgr)
Lowbu,Upbu = outlinefree(data.bu)
Lowsc,Upsc = outlinefree(data.sc)
Lowsod,Upsod = outlinefree(data.sod)
Lowpot,Uppot = outlinefree(data.pot)
Lowhemo,Uphemo = outlinefree(data.hemo)
Lowpcv,Uppcv = outlinefree(data.pcv)
Lowwc,Upwc = outlinefree(data.wc)
Lowrc,Uprc = outlinefree(data.rc)

In [ ]:
data.age.replace(list(data[(data.age < Lowage)].age),Lowage , inplace=True)
data.age.replace(list(data[(data.age > Upage)].age),Upage , inplace=True)

data.bp.replace(list(data[(data.bp < Lowbp)].bp),Lowbp , inplace=True)
data.bp.replace(list(data[(data.bp > Upbp)].bp),Upbp , inplace=True)

data.sg.replace(list(data[(data.sg < Lowsg)].sg),Lowsg , inplace=True)
data.sg.replace(list(data[(data.sg > Upsg)].sg),Upsg , inplace=True)

data.al.replace(list(data[(data.al < Lowal)].al),Lowal , inplace=True)
data.al.replace(list(data[(data.al > Upal)].al),Upal , inplace=True)

data.su.replace(list(data[(data.su < Lowsu)].su),Lowsu , inplace=True)
data.su.replace(list(data[(data.su > Upsu)].su),Upsu , inplace=True)

data.bgr.replace(list(data[(data.bgr < Lowbgr)].bgr),Lowbgr , inplace=True)
data.bgr.replace(list(data[(data.bgr > Upbgr)].bgr),Upbgr , inplace=True)

data.bu.replace(list(data[(data.bu < Lowbu)].bu),Lowbu , inplace=True)
data.bu.replace(list(data[(data.bu > Upbu)].bu),Upbu , inplace=True)

data.sc.replace(list(data[(data.sc < Lowsc)].sc),Lowbu , inplace=True)
data.sc.replace(list(data[(data.sc > Upsc)].sc),Upbu , inplace=True)

data.sod.replace(list(data[(data.sod < Lowsod)].sod),Lowsod , inplace=True)
data.sod.replace(list(data[(data.sod > Upsod)].sod),Upsod , inplace=True)

data.pot.replace(list(data[(data.pot < Lowpot)].pot),Lowpot , inplace=True)
data.pot.replace(list(data[(data.pot > Uppot)].pot),Uppot , inplace=True)

data.hemo.replace(list(data[(data.hemo < Lowhemo)].hemo),Lowhemo , inplace=True)
data.hemo.replace(list(data[(data.hemo > Uphemo)].hemo),Uphemo , inplace=True)

data.pcv.replace(list(data[(data.pcv < Lowpcv)].pcv),Lowpcv , inplace=True)
data.pcv.replace(list(data[(data.pcv > Uppcv)].pcv),Uppcv , inplace=True)

data.wc.replace(list(data[(data.wc < Lowwc)].wc),Lowwc , inplace=True)
data.wc.replace(list(data[(data.wc > Upwc)].wc),Upwc , inplace=True)

data.rc.replace(list(data[(data.rc < Lowrc)].rc),Lowrc , inplace=True)
data.rc.replace(list(data[(data.rc > Uprc)].rc),Uprc , inplace=True)

In [ ]:
data_final_num2 = data[['age','bp','sg','al','su','bgr','bu','sc','sod','pot','hemo','pcv','wc','rc']]

In [ ]:
for i in data_final_num2.columns:
    sns.boxplot(data_final_num2[i])
    plt.title(i)
    plt.show()

## Feature Selection

1. seaborn.pairplot()

In [ ]:
sns.pairplot(data, hue="classification",corner=True)

In [ ]:
finaldata = data.loc[:,['classification','age','bp','sg','al','rbc','pc','pcc','ba','bgr','bu','sc','sod','pot','hemo','pcv','wc','rc','htn','dm','cad','appet','pe','ane']]

## Feature Engineering

1. get_dummies()

In [ ]:
final_dataset = pd.get_dummies(finaldata)

In [ ]:
features = final_dataset.iloc[:,1:].values
label = final_dataset.iloc[:,0].values

## Model Buliding
here we will be using many algorithms and compare all of them. which algorithm will be giving us a Better result. The following algorithms are below.

1. Logistic Regression (f1 score: 0.9827586206896551 )
2. k-nearest neighbors (f1 score: 0.8571428571428571 )
3. **naive bayes (f1 score: 1.0)**
4. support vector classification (f1 score: 0.7730061349693252 )
5. **DecisionTreeClassifier (f1 score: 1.0)**
6. **RandomForestClassifier (f1 score: 1.0)**

In [ ]:
#------------------------LogisticRegression-----------------------
X_train, X_test, y_train, y_test= train_test_split(features,label, test_size= 0.25, random_state=112)

classimodel= LogisticRegression()  
classimodel.fit(X_train,y_train)
trainscore =  classimodel.score(X_train,y_train)
testscore =  classimodel.score(X_test,y_test)  

y_pred =  classimodel.predict(X_test)
print(' f1 score: ',f1_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

In [ ]:
print(' precision score: ',precision_score(y_test, y_pred),'\n')
print(' recall score: ',recall_score(y_test, y_pred),'\n')
print(classification_report(y_test, y_pred))

In [ ]:
#------------------------k-nearest neighbors (K-nn)-----------------------
X_train, X_test, y_train, y_test= train_test_split(features,label, test_size= 0.25, random_state=43)

Knnmodel= KNeighborsClassifier()  
Knnmodel.fit(X_train, y_train)
trainscore =  Knnmodel.score(X_train,y_train)
testscore =  Knnmodel.score(X_test,y_test)  

y_pred =  Knnmodel.predict(X_test)
print(' f1 score: ',f1_score(y_test, y_pred),'\n')
print(confusion_matrix(y_test, y_pred))

In [ ]:
print(' precision score: ',precision_score(y_test, y_pred),'\n')
print(' recall score: ',recall_score(y_test, y_pred),'\n')
print(classification_report(y_test, y_pred))

In [ ]:
#------------------------naive bayes-----------------------
X_train, X_test, y_train, y_test= train_test_split(features,label, test_size= 0.25, random_state=82)

NBmodel=  GaussianNB()  
NBmodel.fit(X_train, y_train)
trainscore =  NBmodel.score(X_train,y_train)
testscore =  NBmodel.score(X_test,y_test)  

y_pred =  NBmodel.predict(X_test)
print(' f1 score: ',f1_score(y_test, y_pred),'\n')
print(confusion_matrix(y_test, y_pred))

In [ ]:
print(' precision score: ',precision_score(y_test, y_pred),'\n')
print(' recall score: ',recall_score(y_test, y_pred),'\n')
print(classification_report(y_test, y_pred))

In [ ]:
#------------------------support vector classification-----------------------
X_train, X_test, y_train, y_test= train_test_split(features,label, test_size= 0.25, random_state=187)

SVCmodel=  SVC(probability=True)  
SVCmodel.fit(X_train, y_train)
trainscore =  SVCmodel.score(X_train,y_train)
testscore =  SVCmodel.score(X_test,y_test)  

y_pred =  SVCmodel.predict(X_test)
print(' f1 score: ',f1_score(y_test, y_pred),'\n')
print(confusion_matrix(y_test, y_pred))

In [ ]:
print(' precision score: ',precision_score(y_test, y_pred),'\n')
print(' recall score: ',recall_score(y_test, y_pred),'\n')
print(classification_report(y_test, y_pred))

In [ ]:
#------------------------Decision Tree-----------------------
X_train, X_test, y_train, y_test= train_test_split(features,label, test_size= 0.25, random_state=78)

DTmodel=  DecisionTreeClassifier(max_depth=3)  
DTmodel.fit(X_train, y_train)
trainscore =  DTmodel.score(X_train,y_train)
testscore =  DTmodel.score(X_test,y_test)  

y_pred =  DTmodel.predict(X_test)
print(' f1 score: ',f1_score(y_test, y_pred),'\n')
print(confusion_matrix(y_test, y_pred))

In [ ]:
print(' precision score: ',precision_score(y_test, y_pred),'\n')
print(' recall score: ',recall_score(y_test, y_pred),'\n')
print(classification_report(y_test, y_pred))

In [ ]:
#------------------------Random Forest-----------------------
X_train, X_test, y_train, y_test= train_test_split(features,label, test_size= 0.25, random_state=110)

RFmodel=  RandomForestClassifier(criterion='entropy',max_depth=3) 
RFmodel.fit(X_train, y_train)
trainscore =  RFmodel.score(X_train,y_train)
testscore =  RFmodel.score(X_test,y_test)  

y_pred =  RFmodel.predict(X_test)
print(' f1 score: ',f1_score(y_test, y_pred),'\n')
print(confusion_matrix(y_test, y_pred))

In [ ]:
print(' precision score: ',precision_score(y_test, y_pred),'\n')
print(' recall score: ',recall_score(y_test, y_pred),'\n')
print(classification_report(y_test, y_pred))

## Receiver Operating Characteristic Curve (ROC AUC)

here we will be using many algorithms and compare all of them. which algorithm will be giving us a Better result. The following algorithms are below.

1. Logistic Regression (auc: 0.9997333333333333)
2. k-nearest neighbors (auc: 0.8948933333333333)
3. **naive bayes (auc: 1.0)**
4. support vector classification (auc: 0.7623200000000001)
5. DecisionTreeClassifier (auc: 0.99892)
6. RandomForestClassifier (auc: 0.9998933333333333)

In [ ]:
#-------------------------------------- LogisticRegression -------------------------------------
probabilityValues = classimodel.predict_proba(features)[:,1]
#Calculate AUC
auc = roc_auc_score(label,probabilityValues)
print(auc)
#Calculate roc_curve
fpr,tpr, threshold =  roc_curve(label,probabilityValues)
plt.plot([0,1],[0,1], linestyle = '--')
plt.plot(fpr,tpr)

In [ ]:
#------------------------k-nearest neighbors (K-nn)-----------------------
probabilityValues = Knnmodel.predict_proba(features)[:,1]
#Calculate AUC
auc = roc_auc_score(label,probabilityValues)
print(auc)
#Calculate roc_curve
fpr,tpr, threshold =  roc_curve(label,probabilityValues)
plt.plot([0,1],[0,1], linestyle = '--')
plt.plot(fpr,tpr)

In [ ]:
#-------------------------------------- naive bayes -------------------------------------
probabilityValues = NBmodel.predict_proba(features)[:,1]
#Calculate AUC
auc = roc_auc_score(label,probabilityValues)
print(auc)
#Calculate roc_curve
fpr,tpr, threshold =  roc_curve(label,probabilityValues)
plt.plot([0,1],[0,1], linestyle = '--')
plt.plot(fpr,tpr)

In [ ]:
#-------------------------------------- support vector classification -------------------------------------
probabilityValues = SVCmodel.predict_proba(features)[:,1]
#Calculate AUC
auc = roc_auc_score(label,probabilityValues)
print(auc)
#Calculate roc_curve
fpr,tpr, threshold =  roc_curve(label,probabilityValues)
plt.plot([0,1],[0,1], linestyle = '--')
plt.plot(fpr,tpr)

In [ ]:
#-------------------------------------- Decision Tree -------------------------------------
probabilityValues = DTmodel.predict_proba(features)[:,1]
#Calculate AUC
auc = roc_auc_score(label,probabilityValues)
print(auc)
#Calculate roc_curve
fpr,tpr, threshold =  roc_curve(label,probabilityValues)
plt.plot([0,1],[0,1], linestyle = '--')
plt.plot(fpr,tpr)

In [ ]:
#-------------------------------------- Random Forest -------------------------------------
probabilityValues = RFmodel.predict_proba(features)[:,1]
#Calculate AUC
auc = roc_auc_score(label,probabilityValues)
print(auc)
#Calculate roc_curve
fpr,tpr, threshold =  roc_curve(label,probabilityValues)
plt.plot([0,1],[0,1], linestyle = '--')
plt.plot(fpr,tpr)

## Conclusion
I will choose a **naive bayes algorithm** for this dataset.

**naive bayes score**

1. **f1_score: 1.0**
2. **auc: 1.0**